<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/%EC%9B%90_%ED%95%AB_%EC%9D%B8%EC%BD%94%EB%94%A9%2C_%EC%A7%80%EB%8B%88%EA%B3%84%EC%88%98%2C_%EB%A1%9C%EB%A0%8C%EC%B8%A0_%EA%B3%A1%EC%84%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


0. 구글드라이브 연결



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = "/content/drive/My Drive/sjcu/porto-seguro-safe-driver-prediction/"

1. 데이터 불러오기

In [ ]:
import pandas as pd

train = pd.read_csv(data_path + 'train.csv', index_col = 'id')
test = pd.read_csv(data_path + 'test.csv', index_col = 'id')
submission = pd.read_csv(data_path +'sample_submission.csv', index_col = 'id')

2. 피처엔지니어링

In [ ]:
all_data = pd.concat([train, test], ignore_index = True) #train, test 데이터 셋 병합
all_data = all_data.drop('target', axis = 1) #타깃값 제거

In [ ]:
all_features = all_data.columns #전체 피처(컬럼)
all_features

Index(['ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01',
       'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
       'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11',
       'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin',
       'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin',
       'ps_calc_20_bin'],
      dtype='obj

3. 원핫인코딩(One-hot Encoding)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_features = [feature for feature in all_features if 'cat' in feature]
onehot_encoder = OneHotEncoder() #원핫인코더 객체 생성
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

In [ ]:
encoded_cat_matrix

<1488028x184 sparse matrix of type '<class 'numpy.float64'>'
	with 20832392 stored elements in Compressed Sparse Row format>

In [ ]:
for feature in all_features:#for문으로 변수 선택
  if 'cat' in feature:
    print(feature)

ps_ind_02_cat
ps_ind_04_cat
ps_ind_05_cat
ps_car_01_cat
ps_car_02_cat
ps_car_03_cat
ps_car_04_cat
ps_car_05_cat
ps_car_06_cat
ps_car_07_cat
ps_car_08_cat
ps_car_09_cat
ps_car_10_cat
ps_car_11_cat


In [ ]:
drop_features = ['ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin',
                 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14']

In [ ]:
remaining_features = [feature for feature in all_features
                      if('cat' not in feature and
                         'calc' not in feature and
                         feature not in drop_features)]

In [ ]:
remaining_features

['ps_ind_01',
 'ps_ind_03',
 'ps_ind_06_bin',
 'ps_ind_07_bin',
 'ps_ind_08_bin',
 'ps_ind_09_bin',
 'ps_ind_15',
 'ps_ind_16_bin',
 'ps_ind_17_bin',
 'ps_ind_18_bin',
 'ps_reg_01',
 'ps_reg_02',
 'ps_reg_03',
 'ps_car_11',
 'ps_car_12',
 'ps_car_13',
 'ps_car_15']

In [ ]:
from scipy import sparse

all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data[remaining_features]),
                               encoded_cat_matrix],
                              format = 'csr')

In [ ]:
num_train = len(train) #

#훈련데이터와 테스트 데이터 나누기
X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values



정규화 지니계수

In [ ]:
import numpy as np

def eval_gini(y_true, y_pred): #실제값과 예측값의 크기가 서로 같은지 확인(값이 다르면 오류 발생)
  assert y_true.shape == y_pred.shape

  n_samples = y_true.shape[0] #데이터 개수
  L_mid = np.linspace(1/n_samples, 1, n_samples) #대각선 값

  pred_order = y_true[y_pred.argsort()] #y_pred크기순으로 y _true 값 정렬

  #1) 예측값에 대한 지니계수
  pred_order = y_true[y_pred.argsort()] #y_pred 크기순으로 y_true값 정렬
  L_pred = np.cumsum(pred_order) / np.sum(pred_order) #로렌츠 곡선
  G_pred = np.sum(L_mid - L_pred) #예측값에 대한 지니계수

  #2)예측이 완벽할 때 지니계수
  true_order = y_true[y_true.argsort()] #y_true 크기순으로 y_true 값 정렬
  L_true = np.cumsum(true_order) / np.sum(true_order) #로렌츠곡선
  G_true = np.sum(L_mid - L_true)

  #정규화된 지니계수
  return G_pred / G_true



LightGBM용 gini()함수

In [ ]:
def gini(preds, dtrain):
  labels = dtrain.get_label()
  return 'gini', eval_gini(labels, preds), True #반환값

OOF방식으로 LightGBM 훈련

In [ ]:
from sklearn.model_selection import StratifiedKFold

#층화 K폴드 교차 검증기
folds = StratifiedKFold(n_splits=5, shuffle =True, random_state=1991)

In [ ]:
params = {'objective': 'binary',
          'learning rate': 0.01,
          'force_row_wise': True,
          'random_state': 0}

In [ ]:
#OOF방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds = np.zeros(X.shape[0])
#OOF방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds = np.zeros(X_test.shape[0])



In [ ]:
import lightgbm as lgb

#OOF방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X,y)):
  #각 폴드를 구분하는 문구 출력
  print('#' *40, f'폴드 {idx +1} / 폴드{folds.n_splits}', '#' * 40)

#훈련용 데이터, 검증용 데이터 설정
  X_train, y_train = X[train_idx], y[train_idx] #훈련용 데이터
  X_valid, y_valid = X[valid_idx], y[valid_idx] #검증용 데이터

  dtrain = lgb.Dataset(X_train, y_train) #LightGBM 전용 훈련 데이터 셋
  dvalid = lgb.Dataset(X_valid, y_valid) #LightGBM 전용 검증 데이터 셋

#LightGBM 모델 훈련
  lgb_model = lgb.train(params = params,
                        train_set = dtrain,
                        num_boost_round = 1000,
                        valid_sets = dvalid,
                        feval = gini,
                        verbose_eval = 500) # 500번째마다 점수 출력

  off_test_preds =+ ...
